In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Курсовой проект для курса "Библиотеки Python для Data Science: Numpy, Matplotlib, Scikit-learn"**

**Тема проекта: "Предсказание цен на жилые дома"**
Материалы к проекту (файлы): train.csv, test.csv

Задание: Используя данные из train.csv, построить модель для предсказания цен на недвижимость (квартиры). С помощью полученной модели предсказать цены для квартир из файла test.csv.

Целевая переменная: Price.

Основная метрика: R2 - коэффициент детерминации (sklearn.metrics.r2_score).

Вспомогательная метрика: MSE - средняя квадратичная ошибка (sklearn.metrics.mean_squared_error).

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

from sklearn import ensemble
from matplotlib import style
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor


from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score

style.use('fivethirtyeight')
%matplotlib inline

**Чтение данных**

In [6]:
TEST_DATASET_PATH = '../input/real-estate-price-prediction-moscow/test.csv'
TRAIN_DATASET_PATH = '../input/real-estate-price-prediction-moscow/train.csv'

X_final = pd.read_csv(TEST_DATASET_PATH)
X = pd.read_csv(TRAIN_DATASET_PATH)

X.info()

**1. Минимальный EDA**

**1.1 Выбросы**

Rooms

In [7]:
plt.figure(figsize=(12, 6))
sns.boxplot(X['Rooms'], X['Price'])
plt.xlabel('Rooms')
plt.ylabel('Price')
plt.title('Distribution of Price by Rooms')
plt.show();

**Square**

In [8]:
var = 'Square'
data = pd.concat([X['Price'], X[var]], axis=1)
data.plot.scatter(x=var, y='Price', ylim=(0,800000));

**LifeSquare**

In [9]:
var = 'LifeSquare'
data = pd.concat([X['Price'], X[var]], axis=1)
data.plot.scatter(x=var, y='Price', ylim=(0,800000));

**Floor, HouseFloor**

In [10]:
X['HouseFloor'].sort_values().unique()

In [11]:
X['Floor'].sort_values().unique()

In [12]:
(X['Floor'] > X['HouseFloor']).sum()

**HouseYear**

In [13]:
X['HouseYear'].sort_values().unique()

**1.2 Анализ количественных признаков**

In [14]:
# отбираем количественные признаки
df_num_features = X.select_dtypes(include='float64')
df_num_features

In [15]:
# Удаляем малозначащие столбцы
num_features = pd.DataFrame(df_num_features)
num_features.drop("Ecology_1", axis=1, inplace=True)
num_features.drop("Healthcare_1", axis=1, inplace=True)
num_features.hist(figsize=(10, 8), bins=20, grid=False);

**1.3 Корреляция признаков**

In [16]:
corr = num_features.corr()
plt.figure(figsize = (8, 8))
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
sns.set(font_scale=1.4)
sns.heatmap(num_features.corr(), mask=mask, annot=True, fmt='.1f', linewidths=.5, cmap='GnBu')
plt.title('Correlation matrix')
plt.show();

In [17]:
X_final.info()

**2. Очистка данных**

In [18]:
X = X.sort_values('Price')
X_final = X_final.sort_values('DistrictId')

In [21]:
# Проверяем выбросы Square
X.sort_values(by = 'Square', ascending = False)[:10]

In [22]:
# Удаляем очевидные выбросы Square
X = X.drop(X[X['Id'] == 11602].index)
X = X.drop(X[X['Id'] == 28].index)
X = X.drop(X[X['Id'] == 2307].index)

In [23]:
# Проверяем выбросы Square
X.sort_values(by = 'Square', ascending = False)[:10]

In [27]:
# Проверяем выбросы HouseYear
X.sort_values(by = 'HouseYear', ascending = False)[:5]

In [28]:
# Удаляем очевидные выбросы HouseYear
X = X.drop(X[X['Id'] == 10814].index)
X = X.drop(X[X['Id'] == 11607].index)

In [29]:
# Проверяем выбросы HouseYear
X.sort_values(by = 'HouseYear', ascending = False)[:5]

In [31]:
# Проверяем выбросы Rooms
X.sort_values(by = 'Rooms', ascending = False)[:5]

In [32]:
# Удаляем очевидные выбросы Rooms
X = X.drop(X[X['Id'] == 8491].index)
X = X.drop(X[X['Id'] == 14865].index)
X = X.drop(X[X['Id'] == 5927].index)

In [33]:
# Проверяем выбросы Rooms
X.sort_values(by = 'Rooms', ascending = False)[:5]

In [34]:
# Проверяем выбросы LifeSquare
X.sort_values(by = 'LifeSquare', ascending = False)[:10]

In [35]:
# Удаляем очевидные выбросы Rooms
X = X.drop(X[X['Id'] == 16550].index)
X = X.drop(X[X['Id'] == 15886].index)
X = X.drop(X[X['Id'] == 14990].index)
X = X.drop(X[X['Id'] == 5548].index)
X = X.drop(X[X['Id'] == 7917].index)
X = X.drop(X[X['Id'] == 16568].index)

In [36]:
# Проверяем выбросы LifeSquare
X.sort_values(by = 'LifeSquare', ascending = False)[:10]

In [37]:
(X['LifeSquare'] > X['Square']).sum()

In [38]:
# Создаем класс для подчищения выбросов


class FeatureImputer:
    """Заполнение пропусков и обработка выбросов"""
    
    def __init__(self):
        self.medians=None
        
    def fit(self, X):
        self.medians = X.median()
    
    def transform(self, X):
        
        # Rooms
        X['Rooms_outlier'] = 0
        X.loc[(X['Rooms'] == 0) | (X['Rooms'] >= 6), 'Rooms_outlier'] = 1
        
        X.loc[X['Rooms'] == 0, 'Rooms'] = 1
        X.loc[X['Rooms'] >= 6, 'Rooms'] = self.medians['Rooms'] 
        X.loc[(X['Rooms'] > 5) & (X['Square'] < 50),'Rooms'] = 2
        
        # Square
        X.loc[X['Square'] < 10,'Square']= X.loc[X['Square'] < 10,'Square']*10
        X.loc[X['Square'] < 1,'Square'] = X.loc[X['Square']<1,'Square']*100 
        X.loc[X['Square'] > 250, 'Square'] = self.medians['Square'] 
        
        # KitchenSquare
        X.loc[X['KitchenSquare'] < 3, 'KitchenSquare'] = 3
        X.loc[X['KitchenSquare'] > 1000, 'KitchenSquare'] = self.medians['KitchenSquare']
        
        # HouseFloor, Floor
        X['HouseFloor_outlier'] = 0
        X.loc[X['HouseFloor'] == 0, 'HouseFloor_outlier'] = 1
        X.loc[X['Floor'] > X['HouseFloor'], 'HouseFloor_outlier'] = 1
        
        X.loc[X['HouseFloor'] == 0, 'HouseFloor'] = self.medians['HouseFloor']
        X.loc[X['Floor'] > X['HouseFloor'], 'Floor'] = X.loc[X['Floor'] > X['HouseFloor'], 'HouseFloor']
        
        # HouseYear
        current_year = now = datetime.datetime.now().year
        
        X['HouseYear_outlier'] = 0
        X.loc[X['HouseYear'] > current_year, 'HouseYear_outlier'] = 1
        
        X.loc[X['HouseYear'] > current_year, 'HouseYear'] = self.medians['HouseYear']
        
        # Healthcare_1
        if 'Healthcare_1' in X.columns:
            X.drop('Healthcare_1', axis=1, inplace=True)
            
        # LifeSquare
        X['LifeSquare_nan'] = X['LifeSquare'].isna() * 1
        
        X.loc[(X['LifeSquare'] > 200) &\
               (X['Square'] < 100),'LifeSquare']=X.loc[(X['LifeSquare']>200) & (X['Square']<100),'LifeSquare']/10
        
        condition = (X['LifeSquare'].isna()) &\
                      (~X['Square'].isna()) & \
                      (~X['KitchenSquare'].isna())
        
        X.loc[condition, 'LifeSquare'] = X.loc[condition, 'Square'] * 0.65
        
        
        return X

In [39]:
imputer = FeatureImputer()

imputer.fit(X) # расчет медиан

X = imputer.transform(X)
X_final = imputer.transform(X_final)

In [40]:
# Заполняем NaN

X = X.sort_values('Price')
X_final = X_final.sort_values('DistrictId')
X = X.fillna(method='pad')
X_final = X_final.fillna(method='pad')

In [41]:
y = X.Price
X.drop("Price", axis=1, inplace=True)
X.drop("Id", axis=1, inplace=True)

**3. Проверка данных**

In [42]:
from pylab import rcParams
rcParams['figure.figsize'] = 12, 6

plt.scatter(y, X.Square)

**4. Модель**

In [43]:
X = pd.get_dummies(X)

In [44]:
model1 = ensemble.GradientBoostingRegressor(n_estimators=300, max_depth=6, min_samples_split=2,
                                           learning_rate=0.1, loss='ls', random_state=42)
cv_score = cross_val_score(model1, X, y, 
                           scoring='r2', 
                           cv=KFold(n_splits=5, shuffle=True, random_state=42))
# cv_score
mean = cv_score.mean()
std = cv_score.std()

print('R2: {:.3f} +- {:.3f}'.format(mean, std))

In [45]:
model2 = RandomForestRegressor(n_estimators=300, max_depth=18, random_state=42, max_features=7)

cv_score = cross_val_score(model2, X, y, 
                           scoring='r2', 
                           cv=KFold(n_splits=5, shuffle=True, random_state=42))
# cv_score
mean = cv_score.mean()
std = cv_score.std()

print('R2: {:.3f} +- {:.3f}'.format(mean, std))

In [46]:
model3 = LGBMRegressor(max_depth=6,
                             min_samples_leaf=10,
                             n_estimators=300,
                             random_state=42)

cv_score = cross_val_score(model3, X, y, 
                           scoring='r2', 
                           cv=KFold(n_splits=5, shuffle=True, random_state=42))
# cv_score
mean = cv_score.mean()
std = cv_score.std()

print('R2: {:.3f} +- {:.3f}'.format(mean, std))

In [47]:
model = VotingRegressor([('model1', model1), ('model2', model2), ('model3', model3)])

In [48]:
cv_score = cross_val_score(model, X, y, 
                           scoring='r2', 
                           cv=KFold(n_splits=5, shuffle=True, random_state=42))
# cv_score
mean = cv_score.mean()
std = cv_score.std()

print('R2: {:.3f} +- {:.3f}'.format(mean, std))

In [49]:
model.fit(X, y)

**5. Предсказание и выгрузка в файл**

In [50]:
X_test = pd.get_dummies(X_final)
X_test.drop("Id", axis=1, inplace=True)



X_final["Price"] = model.predict(X_test)
X_final.loc[:, ['Id', 'Price']].to_csv('predictionsVR.csv', index=False)
X_final.loc[:, ['Id', 'Price']].head(20)